In [ ]:
library(httr)
library(jsonlite)

In [ ]:
classify_emotion <- function(api_key, input_text) {
  url <- "https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash-latest:generateContent"
  
  prompt <- sprintf("Given a sentence, classify its emotional tone into one of the following six emotions: sadness (0), joy (1), love (2), anger (3), fear (4), or surprise (5). Return only the corresponding ID of the detected emotion.
  Examples:
  Input: I can\'t believe this happened, I am so frustrated!
  Output: 3
  Input: I am so proud of you. I love you so much.
  Output: 2
  Input: This is amazing news! I am overjoyed!
  Output: 1
  Input: I am scared of what might happen next.
  Output: 4
  Task: Classify the emotion of the following sentence and return only the ID:
  %s", input_text)
  
  body <- list(
    contents = list(
      list(
        parts = list(
          list(text = prompt)
        )
      )
    )
  )
  
  response <- POST(
    url = paste0(url, "?key=", api_key),
    body = toJSON(body, auto_unbox = TRUE),
    add_headers("Content-Type" = "application/json"),
    encode = "json"
  )
  
raw_content <- rawToChar(response$content)

cat("1. Raw Response:", raw_content, "\n\n")

tryCatch({
  content <- fromJSON(raw_content)
  cat("2. Parsed content structure:\n")
  str(content)
  cat("\n")
  
  candidates <- content$candidates
  cat("3. Candidates structure:\n")
  str(candidates)
  cat("\n")
  
  if (!is.null(candidates) && length(candidates) > 0) {
    response_text <- candidates$content$parts[[1]]$text
    cat("4. Raw text value:", response_text, "\n")
    
    cleaned_text <- trimws(response_text)
    cat("5. Cleaned text value:", cleaned_text, "\n")
    
    emotion_id <- as.numeric(cleaned_text)
    cat("6. Final numeric value:", emotion_id, "\n")
    
    if (!is.na(emotion_id)) {
      return(emotion_id)
    } else {
      cat("Error: Không thể chuyển đổi thành số.\n")
      return(NULL)
    }
  } else {
    cat("Error: Không tìm thấy candidates.\n")
    return(NULL)
  }
}, error = function(e) {
  cat("Error processing response:", e$message, "\n")
  return(NULL)
})
}

In [ ]:
api_key <- "YOUR_API_KEY"

In [ ]:
input_data <- read_csv("~/comments.csv")

In [ ]:
total_rows <- nrow(input_data)
cat("Processing", total_rows, "rows\n")

input_data$label <- sapply(seq_len(nrow(input_data)), function(i) {
  if (i %% 10 == 0) {
    cat(sprintf("Processing row %d of %d (%.1f%%)\n", 
                i, total_rows, i/total_rows*100))
  }
  
  result <- classify_emotion(api_key, input_data$text[i])
  
  Sys.sleep(1)
  
  return(result)
})

In [ ]:
write_csv(input_data, "output.csv")
cat("\nProcessing completed. Results saved to output.csv\n")

cat("\nEmotion Distribution:\n")
table(input_data$label)